In [17]:
import pandas as pd
import numpy as np
import xgboost as xgb

from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer


In [20]:
df_train = pd.read_csv('../data/train_processed.csv')

In [21]:
df_train

,LTFRONT,LTDEPTH,STORIES,FULLVAL,AVLAND,AVTOT,EXLAND,EXTOT,EXMPTCL,BLDFRONT,...,TAXCLASS_1,TAXCLASS_1A,TAXCLASS_1B,TAXCLASS_1C,TAXCLASS_2,TAXCLASS_2A,TAXCLASS_2B,TAXCLASS_2C,TAXCLASS_3,TAXCLASS_4
0,-1.571508,-1.609389,0.993085,184765,0.138225,0.317049,-0.881100,-0.982121,0,-1.475296,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,0.312096,0.500529,-0.237693,556000,-0.125038,-0.304156,-0.881100,-0.982121,0,0.257734,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.135864,0.395352,-0.237693,318000,-0.553620,-0.588107,1.137090,1.067406,0,0.028720,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-1.571508,-1.609389,2.213655,222378,0.000970,0.419364,-0.881100,1.150025,0,1.329968,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.051219,0.500529,-0.237693,1196000,0.011665,-0.242866,1.132417,0.849311,0,0.223410,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8357,0.176482,0.550847,-0.237693,396000,-0.277826,-0.383232,-0.881100,-0.982121,0,0.071258,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8358,0.051219,0.500529,-0.237693,1598000,0.011025,-0.152870,1.004659,0.705081,0,0.223410,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8359,0.260211,0.500529,0.469486,294416,-0.944408,-0.040217,-0.881100,-0.982121,0,-1.475296,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
8360,1.627374,0.712272,1.591373,264968,0.534135,0.517706,1.412057,1.415971,0,-1.475296,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [29]:
df_train.columns[19:]

Index(['BLDGCL_Family', 'BLDGCL_Industrial', 'BLDGCL_Multifamily',
       'BLDGCL_Public use', 'BLDGCL_Service', 'EXT_E', 'EXT_EG', 'EXT_G',
       'EXT_No', 'EASEMENT_E',
       ...
       'TAXCLASS_1', 'TAXCLASS_1A', 'TAXCLASS_1B', 'TAXCLASS_1C', 'TAXCLASS_2',
       'TAXCLASS_2A', 'TAXCLASS_2B', 'TAXCLASS_2C', 'TAXCLASS_3',
       'TAXCLASS_4'],
      dtype='object', length=106)

,Longitude,Latitude
0,-73.991172,40.692171
1,-73.894848,40.824607
2,-74.167801,40.618222
3,-74.016151,40.706358
4,-74.013429,40.640613
...,...,...
8357,-74.152527,40.624109
8358,-74.006687,40.639257
8359,-73.981846,40.658670
8360,-74.016574,40.710118


In [65]:
df_train = pd.read_csv('../data/train_processed.csv')
df_val = pd.read_csv("../data/val_processed.csv")
train_y = df_train['FULLVAL']
val_y = df_val['FULLVAL']
cat = ['BLDGCL', 'EXT', 'EASEMENT', 'EXCD2']
columns_to_delete = [
        'BBLE',
        'POSTCODE',
        'PERIOD',
        'YEAR',
        'VALTYPE',
        'Borough',
        'Community Board',
        'BORO',
        'Council District',
        'Census Tract',
        'BIN',
        'NTA',
        'New Georeferenced Column',
        'FULLVAL',
        'BLOCK',
        *cat,
        'TAXCLASS'
    ]

df_train = df_train.drop(columns=columns_to_delete)
df_val = df_val.drop(columns=columns_to_delete)
df_train = df_train.dropna(axis=1)
df_val = df_val.dropna(axis=1)

In [67]:
# df_train[['BLDGCL', 'TAXCLASS']] = df_train[['BLDGCL', 'TAXCLASS']].astype('category')
# df_val[['BLDGCL', 'TAXCLASS']] = df_val[['BLDGCL', 'TAXCLASS']].astype('category')

In [79]:
import xgboost as xgb

dtrain = xgb.DMatrix(df_train.values, enable_categorical=True, label=train_y, feature_names=df_train.columns)
dval = xgb.DMatrix(df_val.values, enable_categorical=True, label=val_y, feature_names=df_val.columns)
results={}

In [82]:
param = {'objective':'reg:squarederror','eval_metric':'rmse', 'verbosity':2, 'nthread':2}
reg = xgb.train(param,dtrain, 300, evals=[(dval, 'val')], evals_result=results)

[21:14:00] INFO: /Users/runner/work/xgboost/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 30 extra nodes, 0 pruned nodes, max_depth=6
[0]	val-rmse:5281421.00000
[21:14:00] INFO: /Users/runner/work/xgboost/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 30 extra nodes, 0 pruned nodes, max_depth=6
[1]	val-rmse:3924318.75000
[21:14:00] INFO: /Users/runner/work/xgboost/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 26 extra nodes, 0 pruned nodes, max_depth=6
[2]	val-rmse:3043656.75000
[21:14:00] INFO: /Users/runner/work/xgboost/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 28 extra nodes, 0 pruned nodes, max_depth=6
[3]	val-rmse:2392441.00000
[21:14:00] INFO: /Users/runner/work/xgboost/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 28 extra nodes, 0 pruned nodes, max_depth=6
[4]	val-rmse:1876814.00000
[21:14:00] INFO: /Users/runner/work/xgboost/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 28 extra nodes, 0 pruned nodes, max_dep

In [87]:
min(results['val']['rmse'])

747655.3125